## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-24-14-53-41 +0000,nyt,"Michael Whatley, R.N.C. Chairman, to Run for S...",https://www.nytimes.com/2025/07/24/us/politics...
1,2025-07-24-14-53-34 +0000,nypost,"Dozens injured, including child, in Russian st...",https://nypost.com/2025/07/24/world-news/dozen...
2,2025-07-24-14-49-10 +0000,nypost,NJ Transit bus crashes with another bus outsid...,https://nypost.com/2025/07/24/us-news/nj-trans...
3,2025-07-24-14-46-09 +0000,nyt,Live Updates: Thailand Says 12 Killed in Borde...,https://www.nytimes.com/live/2025/07/24/world/...
4,2025-07-24-14-45-55 +0000,nyt,Trump to Visit Federal Reserve As Campaign Aga...,https://www.nytimes.com/2025/07/24/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2324/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
38,trump,54
14,new,21
71,trade,18
231,russia,14
411,fire,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
186,2025-07-24-01-00-00 +0000,wsj,President Trump’s push to introduce a new stan...,https://www.wsj.com/economy/trade/trump-trade-...,136
252,2025-07-23-21-45-42 +0000,nypost,Third round of Ukraine-Russia peace talks end ...,https://nypost.com/2025/07/23/us-news/ukraine-...,131
215,2025-07-23-23-26-00 +0000,wsj,The Trump administration issued a new AI actio...,https://www.wsj.com/tech/ai/trump-pledges-move...,111
325,2025-07-23-18-03-36 +0000,nyt,How Japan Clinched a Trade Deal With Trump Jus...,https://www.nytimes.com/2025/07/23/business/tr...,110
272,2025-07-23-20-52-06 +0000,latimes,Trump can fire the 3 Democrats on the Consumer...,https://www.latimes.com/politics/story/2025-07...,102


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
186,136,2025-07-24-01-00-00 +0000,wsj,President Trump’s push to introduce a new stan...,https://www.wsj.com/economy/trade/trump-trade-...
252,77,2025-07-23-21-45-42 +0000,nypost,Third round of Ukraine-Russia peace talks end ...,https://nypost.com/2025/07/23/us-news/ukraine-...
96,71,2025-07-24-10-57-44 +0000,nypost,Bryan Kohberger will spend the rest of his lif...,https://nypost.com/2025/07/24/us-news/universi...
138,48,2025-07-24-07-33-58 +0000,nypost,Thailand and Cambodia exchange fire in new bor...,https://nypost.com/2025/07/24/world-news/thail...
55,47,2025-07-24-13-18-09 +0000,nyt,House Panel Votes to Subpoena Justice Dept. fo...,https://www.nytimes.com/2025/07/23/us/politics...
206,45,2025-07-23-23-44-11 +0000,nypost,Kamala Harris’ ex-lover Willie Brown warns her...,https://nypost.com/2025/07/23/us-news/kamala-h...
167,41,2025-07-24-03-03-24 +0000,nypost,Appeals court blocks Trump’s effort to end bir...,https://nypost.com/2025/07/23/us-news/appeals-...
28,38,2025-07-24-14-19-43 +0000,nypost,Tesla stock plunges as Elon Musk warns company...,https://nypost.com/2025/07/24/business/tesla-s...
70,37,2025-07-24-12-13-24 +0000,nypost,American Airlines flight attendant who admitte...,https://nypost.com/2025/07/24/us-news/american...
295,37,2025-07-23-19-45-05 +0000,latimes,Tech companies want to move fast. Trump's 'AI ...,https://www.latimes.com/business/story/2025-07...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
